In [24]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score

In [2]:
# import os
# for dirname, _, filenames in os.walk('D:/competition/kaggle/riiid-test-answer-prediction'):
#     print(dirname,'aaa')
#     print(_,'bbb')
#     print(dirname,'ccc')



In [3]:
#遍历该目录下的所有文件，包括子文件夹下的文件
import os
for dirname, _, filenames in os.walk('D:/competition/kaggle/riiid-test-answer-prediction'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

D:/competition/kaggle/riiid-test-answer-prediction\baseline.ipynb
D:/competition/kaggle/riiid-test-answer-prediction\baseline_lgb.ipynb
D:/competition/kaggle/riiid-test-answer-prediction\data-analysis-baseline.ipynb
D:/competition/kaggle/riiid-test-answer-prediction\example_sample_submission.csv
D:/competition/kaggle/riiid-test-answer-prediction\example_test.csv
D:/competition/kaggle/riiid-test-answer-prediction\lectures.csv
D:/competition/kaggle/riiid-test-answer-prediction\questions.csv
D:/competition/kaggle/riiid-test-answer-prediction\test-data-analysis-001.ipynb
D:/competition/kaggle/riiid-test-answer-prediction\train.csv
D:/competition/kaggle/riiid-test-answer-prediction\.ipynb_checkpoints\baseline-checkpoint.ipynb
D:/competition/kaggle/riiid-test-answer-prediction\.ipynb_checkpoints\baseline_lgb-checkpoint.ipynb
D:/competition/kaggle/riiid-test-answer-prediction\baseline\__MACOSX\开源-新手-baseline\._.DS_Store
D:/competition/kaggle/riiid-test-answer-prediction\baseline\__MACOSX\开源-新

In [25]:
nrows = 100 * 10000
# nrows = None

train = pd.read_csv('D:/competition/kaggle/riiid-test-answer-prediction/train.csv',
                   nrows=nrows, 
                   usecols=[1, 2, 3, 4, 7, 8, 9],
                   dtype={'timestamp': 'int64',
                          'user_id': 'int32',
                          'content_id': 'int16',
                          'content_type_id': 'int8',
                          'answered_correctly':'int8',
                          'prior_question_elapsed_time': 'float32',
                          'prior_question_had_explanation': 'boolean'}
                   )

In [5]:
train = train[train.content_type_id == False]#选出答题记录
train = train.sort_values(['timestamp'], ascending=True).reset_index(drop = True)#reset_index把原index变为名为index的列，drop=True就扔掉这一列

In [6]:
#行为准确度统计
results_c_final = train[['content_id','answered_correctly']].groupby(['content_id']).agg(['mean','sum','count','std'])
results_c_final.columns = ["content_y_mean","content_y_sum","content_y_count","content_y_std"]
results_c_final

,content_y_mean,content_y_sum,content_y_count,content_y_std
content_id,,,,
0,0.863014,63.0,73,0.346212
1,0.927273,51.0,55,0.262082
2,0.560811,249.0,444,0.496848
3,0.798995,159.0,199,0.401763
4,0.602606,185.0,307,0.490158
...,...,...,...,...
13518,0.750000,6.0,8,0.462910
13519,0.555556,5.0,9,0.527046
13520,0.700000,7.0,10,0.483046


In [7]:
#用户准确度统计
results_u_final = train[['user_id','answered_correctly']].groupby(['user_id']).agg(['mean', 'sum', 'count','std'])
results_u_final.columns = ['user_y_mean', 'user_y_sum', 'user_y_count','user_y_std']
results_u_final

,user_y_mean,user_y_sum,user_y_count,user_y_std
user_id,,,,
115,0.695652,32.0,46,0.465215
124,0.233333,7.0,30,0.430183
2746,0.578947,11.0,19,0.507257
5382,0.672000,84.0,125,0.471374
8623,0.642202,70.0,109,0.481566
...,...,...,...,...
20913319,0.630653,251.0,398,0.483235
20913864,0.333333,7.0,21,0.483046
20938253,0.608779,319.0,524,0.488490


In [8]:
#行为时间统计
results_ct_final = train[['content_id','prior_question_elapsed_time']].groupby(['content_id']).agg(['mean', 'sum', 'count','std'])
results_ct_final.columns = ['content_t_mean', 'content_t_sum', 'content_t_count','content_t_std']
# results_ct_final

In [9]:
#用户时间统计
results_ut_final = train[['user_id','prior_question_elapsed_time']].groupby(['user_id']).agg(['mean', 'sum', 'count','std'])
results_ut_final.columns = ['user_t_mean', 'user_t_sum', 'user_t_count','user_t_std']
# results_ut_final

In [10]:
train.drop(['timestamp', 'content_type_id'], axis=1, inplace=True)

In [11]:
#循环依次取出每个用户后六次操作
validation = pd.DataFrame()

for i in range(6):
    last_records = train.drop_duplicates('user_id', keep = 'last')
    train = train[~train.index.isin(last_records.index)]#isin()没有反函数，故使用位运算取反~
    validation = validation.append(last_records)
    print('validation : ', len(validation))


validation :  3824
validation :  7646
validation :  11466
validation :  15285
validation :  19104
validation :  22923


In [12]:
#循环依次取出每个用户后30次操作(去掉6行后的)
X = pd.DataFrame()

for i in range(30):
    last_records = train.drop_duplicates('user_id', keep = 'last')
    train = train[~train.index.isin(last_records.index)]
    X = X.append(last_records)
    print('X : ', X.shape)

X :  (3819, 5)
X :  (7626, 5)
X :  (11432, 5)
X :  (15236, 5)
X :  (19034, 5)
X :  (22819, 5)
X :  (26539, 5)
X :  (30175, 5)
X :  (33770, 5)
X :  (37352, 5)
X :  (40901, 5)
X :  (44295, 5)
X :  (47633, 5)
X :  (50909, 5)
X :  (54120, 5)
X :  (57262, 5)
X :  (60369, 5)
X :  (63460, 5)
X :  (66534, 5)
X :  (69597, 5)
X :  (72639, 5)
X :  (75602, 5)
X :  (78542, 5)
X :  (81437, 5)
X :  (83732, 5)
X :  (85963, 5)
X :  (88171, 5)
X :  (90350, 5)
X :  (92512, 5)
X :  (94636, 5)


In [13]:
results_c_tv = train[['content_id','answered_correctly']].groupby(['content_id']).agg(['mean','sum','count','std'])
results_c_tv.columns = ["content_y_mean","content_y_sum","content_y_count","content_y_std"]
results_c_tv

,content_y_mean,content_y_sum,content_y_count,content_y_std
content_id,,,,
0,0.897059,61.0,68,0.306141
1,0.924528,49.0,53,0.266679
2,0.564885,222.0,393,0.496404
3,0.790055,143.0,181,0.408399
4,0.619608,158.0,255,0.486438
...,...,...,...,...
13518,0.800000,4.0,5,0.447214
13519,0.428571,3.0,7,0.534522
13520,0.666667,6.0,9,0.500000


In [14]:
results_u_tv = train[['user_id','answered_correctly']].groupby(['user_id']).agg(['mean', 'sum', 'count','std'])
results_u_tv.columns = ['user_y_mean', 'user_y_sum', 'user_y_count','user_y_std']
results_u_tv

,user_y_mean,user_y_sum,user_y_count,user_y_std
user_id,,,,
115,1.000000,10.0,10,0.000000
5382,0.707865,63.0,89,0.457320
8623,0.616438,45.0,73,0.489618
12741,0.545852,125.0,229,0.498984
13134,0.707539,854.0,1207,0.455081
...,...,...,...,...
20897683,0.683566,391.0,572,0.465491
20913319,0.635359,230.0,362,0.481996
20938253,0.625000,305.0,488,0.484620


In [15]:
results_ct_tv = train[['content_id','prior_question_elapsed_time']].groupby(['content_id']).agg(['mean', 'sum', 'count','std'])
results_ct_tv.columns = ['content_t_mean', 'content_t_sum', 'content_t_count','content_t_std']

results_ut_tv = train[['user_id','prior_question_elapsed_time']].groupby(['user_id']).agg(['mean', 'sum', 'count','std'])
results_ut_tv.columns = ['user_t_mean', 'user_t_sum', 'user_t_count','user_t_std']

In [16]:
del(train)

In [17]:
X = pd.merge(X, results_u_tv, on=['user_id'], how="left")
print(X.columns)
X = pd.merge(X, results_c_tv, on=['content_id'], how="left")
print(X.columns)
X = pd.merge(X, results_ut_tv, on=['user_id'], how="left")
print(X.columns)
X = pd.merge(X, results_ct_tv, on=['content_id'], how="left")
print(X.columns)

Index(['user_id', 'content_id', 'answered_correctly',
       'prior_question_elapsed_time', 'prior_question_had_explanation',
       'user_y_mean', 'user_y_sum', 'user_y_count', 'user_y_std'],
      dtype='object')
Index(['user_id', 'content_id', 'answered_correctly',
       'prior_question_elapsed_time', 'prior_question_had_explanation',
       'user_y_mean', 'user_y_sum', 'user_y_count', 'user_y_std',
       'content_y_mean', 'content_y_sum', 'content_y_count', 'content_y_std'],
      dtype='object')
Index(['user_id', 'content_id', 'answered_correctly',
       'prior_question_elapsed_time', 'prior_question_had_explanation',
       'user_y_mean', 'user_y_sum', 'user_y_count', 'user_y_std',
       'content_y_mean', 'content_y_sum', 'content_y_count', 'content_y_std',
       'user_t_mean', 'user_t_sum', 'user_t_count', 'user_t_std'],
      dtype='object')
Index(['user_id', 'content_id', 'answered_correctly',
       'prior_question_elapsed_time', 'prior_question_had_explanation',
       

In [18]:
validation = pd.merge(validation, results_u_tv, on=['user_id'], how="left")
validation = pd.merge(validation, results_c_tv, on=['content_id'], how="left")
validation = pd.merge(validation, results_ut_tv, on=['user_id'], how="left")
validation = pd.merge(validation, results_ct_tv, on=['content_id'], how="left")

# 这里为什么要对‘是否看了解析’feature做label encode?

In [19]:
from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()

X.prior_question_had_explanation.fillna(False, inplace = True)
validation.prior_question_had_explanation.fillna(False, inplace = True)

validation["prior_question_had_explanation_enc"] = lb_make.fit_transform(validation["prior_question_had_explanation"])
X["prior_question_had_explanation_enc"] = lb_make.fit_transform(X["prior_question_had_explanation"])

In [20]:
#split
y = X['answered_correctly']
X = X.drop(['answered_correctly'], axis=1)

y_val = validation['answered_correctly']
X_val = validation.drop(['answered_correctly'], axis=1)

In [21]:
# X.columns

columns_features = [c for c in X.columns if c not in ['user_id','content_id','prior_question_had_explanation']]

X = X[columns_features]
X_val = X_val[columns_features]

X.fillna(-1,  inplace=True)
X_val.fillna(-1, inplace=True)

In [22]:
import lightgbm as lgb

params = {
    'objective': 'binary',
    'max_bin': 600,
    'learning_rate': 0.02,
    'num_leaves': 80
}

lgb_train = lgb.Dataset(X, y)
lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train)


model = lgb.train(
    params, lgb_train,
    valid_sets=[lgb_train, lgb_eval],
    verbose_eval=25,
    num_boost_round=10000,
    early_stopping_rounds=10
)

Training until validation scores don't improve for 10 rounds
[25]	training's binary_logloss: 0.630124	valid_1's binary_logloss: 0.65008
[50]	training's binary_logloss: 0.606528	valid_1's binary_logloss: 0.635373
[75]	training's binary_logloss: 0.594979	valid_1's binary_logloss: 0.629873
[100]	training's binary_logloss: 0.588382	valid_1's binary_logloss: 0.627565
[125]	training's binary_logloss: 0.583842	valid_1's binary_logloss: 0.626697
[150]	training's binary_logloss: 0.580186	valid_1's binary_logloss: 0.626281
[175]	training's binary_logloss: 0.576758	valid_1's binary_logloss: 0.625979
[200]	training's binary_logloss: 0.573547	valid_1's binary_logloss: 0.625573
[225]	training's binary_logloss: 0.570529	valid_1's binary_logloss: 0.625361
Early stopping, best iteration is:
[235]	training's binary_logloss: 0.569394	valid_1's binary_logloss: 0.625325


In [23]:
y_pred = model.predict(X_val)
y_true = np.array(y_val)
roc_auc_score(y_true, y_pred)

0.70289963479393